# MD simulation of LJ liquid

**`By You-Liang Zhu`**

## Introduction

This tutorial is designed to provide an introduction to MD simulations with GALAMOST. It is assumed that you have already got a basic knowledge of the Linux command line (if not, you can begin with this little learning program: BEGINNER'S CORNER: Learning the [Unix Command-line](https://www.codecademy.com/learn/learn-the-command-line)) and DPD principle before. It is suitable for new users who want to learn about how to run MD simulations. It does however assume that you have a machine with GALAMOST correctly installed.

In this tutorial, we would like to make a system consisting of polymers and solvents, and we want to simulate the phase separation in the system as the temperature is quenching or decreasing.

**Download [running files](https://bitbucket.org/galamostdevelopergroup/source-code/src/master/examples/Case3-DPD-Polymer-Solution/)**.

In order to run this DPD simulation with GALAMOST, two files should be prepared:
*  `xml`: the file that describes all the information (including position, velocity, topology, etc.) of the particles in the system, more about [XML data format](https://galamost.readthedocs.io/en/latest/data-format.html); the xml configuration file could be generated by `Molgen` plugin
*  `gala`: The file that describes the settings for the GALAMOST simulation engine

## Prepare topology and coordinate files

For this tutorial, you will build a system containing two kinds of molecules (shown as below) for simulation.
In order to build the two molecules, you need to write a python script of [`Molgen`](https://galamost.readthedocs.io/en/latest/molgen.html) that will run on a terminal of Linux to call the [`Molgen`](https://galamost.readthedocs.io/en/latest/molgen.html) plugin of GALAMOST.
The script "polymer.molg" to build a polymer-solvent system is shown as below in blue

In [ ]:
import molgen

mol=molgen.Molecule(1)
mol.setParticleTypes("a")

gen=molgen.Generators(20.0, 20.0, 20.0)
gen.addMolecule(mol,6000)
gen.setMinimumDistance(1.0);
gen.outPutMST("lj")

## Prepare GALAMOST DPD gala input files

Next component needed is the input files that define the program settings for each DPD run. For this system, firstly, we will perform an equilibrium run on the system at a high temperature of $k_BT$ = 1.73, then decrease the system temperature to induce phase separation.
The gala file "polymer.gala" is shown as below in blue:

In [ ]:
import gamst

mst = gamst.snapshot.read("lj.mst")
app = gamst.application.dynamics(info=mst, dt=0.001)

fn = gamst.force.nonbonded(info=mst, rcut=3.0, func='lj')
fn.setParams(type_i="a", type_j="a", param=[1.0, 1.0, 1.0, 3.0])
app.add(fn)

inn = gamst.integration.nvt(info=mst, group='all', method="nh", tau=1.0, temperature=1.0)
app.add(inn)

dd = gamst.dump.data(info=mst, group='all', file='data.log', period=100)
app.add(dd)

dm = gamst.dump.mst(info=mst, group='all', file='p.mst', period=10000)
app.add(dm)

app.run(20000)

Simulation of LJ liquid with self-defined function

In [ ]:
import gamst
from numba import cuda
import numba as nb

mst = gamst.snapshot.read("lj.mst")
app = gamst.application.dynamics(info=mst, dt=0.001)

@cuda.jit(device=True)
def lj(rsq, param, fp):
	epsilon = param[0]
	sigma = param[1]
	alpha = param[2]
	rcut = param[3]
	if rsq<rcut*rcut:
		sigma2 = sigma*sigma
		r2inv = sigma2/rsq;
		r6inv = r2inv * r2inv * r2inv;
		f = nb.float32(4.0) * r2inv * r6inv * (nb.float32(12.0) * r6inv - nb.float32(6.0) * alpha)/sigma2	
		p = nb.float32(4.0) * r6inv * ( r6inv - nb.float32(1.0))
		fp[0]=f
		fp[1]=p	

fn = gamst.force.nonbonded(info=mst, rcut=3.0, func=lj)
fn.setParams(type_i="a", type_j="a", param=[1.0, 1.0, 1.0, 3.0])
app.add(fn)

inn = gamst.integration.nvt(info=mst, group='all', method="nh", tau=1.0, temperature=1.0)
app.add(inn)

di = gamst.dump.data(info=mst, group='all', file='data.log', period=100)
app.add(di)

app.run(1000)